<a href="https://colab.research.google.com/github/Muntasir2179/vector-database-learning/blob/main/VD_Pinecone_symentic_search_%2B_metadata_filterting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone - symentic search with metadata filtering

* News articles.
* symbol, publishedDate, title, site

In [1]:
!pip install pinecone-client sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=600c923cb5f747001ba39d4b23e5b48a992a64acbffdb549591b46baa1c81cea
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import time
import pandas as pd
import numpy as np

from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

In [4]:
pc = Pinecone(api_key="af8ce84a-ab95-401c-ba72-b5249afdb37a")

In [5]:
pc.list_indexes()

{'indexes': []}

## Prepare Data - News Articles

In [9]:
df = pd.read_csv('Articles.csv', encoding = "cp1252")
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [12]:
df['NewsType'].value_counts()

sports      1408
business    1284
Name: NewsType, dtype: int64

In [17]:
# removing null values bases on date
df['Date'] = df['Date'].fillna('')
len(df)

2692

## Create Embeddings and Vectors

* We will be using `average_word_embeddings_komninos` sentence-transformer to generate embeddings.

* The embedding dimension will be 300.

In [16]:
model = SentenceTransformer(model_name_or_path='average_word_embeddings_komninos')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

In [18]:
model

SentenceTransformer(
  (0): WordEmbeddings(
    (emb_layer): Embedding(222305, 300)
  )
  (1): Pooling({'word_embedding_dimension': 300, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [21]:
print("Encoding articles....")
encoded_articles = model.encode(df['Article'].tolist(), show_progress_bar=True)

Encoding articles....


Batches:   0%|          | 0/85 [00:00<?, ?it/s]

In [22]:
df['article-encoded'] = encoded_articles.tolist()

In [23]:
df.head()

,Article,Date,Heading,NewsType,article-encoded
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,"[0.23253199458122253, 0.024697260931134224, -0..."
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,"[0.10716943442821503, -0.012101680040359497, -..."
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business,"[-0.04096980020403862, 0.06097852811217308, 0...."
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,"[0.06540685892105103, 0.0188664011657238, 0.01..."
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,"[0.12384109199047089, 0.02100343629717827, 0.0..."
